In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
import os
from IPython.display import display_html
import marko

from taogpt.orchestrator import *
from taogpt.utils import *
from taogpt.llm_model import LangChainLLM
from taogpt.prompts import PromptDb
import taogpt.utils as utils

utils.enable_debugging(0)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
TEMPERATURE = 0.7

In [4]:
with open(os.path.join(os.environ['HOME'], '.ssh', 'openai-zillow.json'), 'r') as f:
    credentials = json.load(f)
os.environ["OPENAI_API_KEY"] = credentials['key']
os.environ["OPENAI_API_BASE"] = credentials['url']
llm3_5 = ChatOpenAI(model_name='gpt-3.5-turbo-16k', temperature=TEMPERATURE)
llm4 = ChatOpenAI(model_name='gpt-4-32k', temperature=TEMPERATURE)

conversation = ConversationChain(llm=llm3_5)
conversation.predict(input="What's your model version?")

'I am an OpenAI GPT-3 model, which is the third iteration of the Generative Pre-trained Transformer model developed by OpenAI.'

In [5]:
llm3_5.model_name, llm4.model_name

('gpt-3.5-turbo-16k', 'gpt-4-32k')

In [17]:
prompts = PromptDb.load_defaults()
logger = MarkdownLogger('logs/taogpt_log.md')
executor = Orchestrator(llm=LangChainLLM(llm4, logger=logger), prompts=prompts, markdown_logger=logger,
                        initial_expansion=1,
                        max_tree_branches=2,
                        max_tokens=10000,
                        check_final=True,
                        # smarter_llm=LangChainLLM(llm4, logger=logger),
                        )

experiment_name = 'example'
executor.start("""
Solve this Game of 24: 7, 6, 2, 1
""", analyze_first=True)

In [16]:
executor.resume(10000)

extend token allowance by 10000 to 20000


RuntimeError: Regular LLM consumed 22718 tokens, exceeded allowance of 20000

In [18]:
logger = MarkdownLogger(f'examples/{experiment_name}.final.md')
logger.log_conversation(executor.show_conversation_thread(with_extras=True))
logger.log(f"**total tokens**: {executor.llm.total_tokens}")

In [19]:
with open(logger._log_path, 'r') as f:
    html_output = marko.convert(f.read())
    display_html(html_output, raw=True)


Final path history 
 
 
 
 [0] user 
 
 
 Solve this Game of 24: 7, 6, 2, 1 
 
 
 
 
 [1] orchestrator 
 
 
 Some critics said this problem is not solvable due to fundamental errors. Other critics said it is solvable. 
 Does the task problem contain any fundamental errors, invalid assumptions, or contradiction in the problem statement? 
 Do not try to solve the problem yet, just provide analysis. 
 
 
 
 
 [2] Tao 
 
 
 The problem statement is clear and does not contain any fundamental errors, invalid assumptions, or contradictions. The task is to use the four given numbers (7, 6, 2, 1) in any combination of addition, subtraction, multiplication, and division to arrive at the number 24. Each number must be used exactly once and the order of operations should be considered. This is a standard format for a Game of 24 problem. 
 
 
 
 
 [3] orchestrator 
 
 
 Tao, please proceed. You must follow the response templates strictly so that your orchestrator
can parse your response. Think carefully before responding. 
 
 
 
 
 [4] Tao 
 
 
 The task requires us to use the four given numbers (7, 6, 2, 1) in any combination of addition, subtraction, multiplication, and division to arrive at the number 24. Each number must be used exactly once and the order of operations should be considered. 
 
 Try different combinations of the four numbers using the four operations. [This step is necessary because the solution could be any combination of the four numbers with the four operations.] 
 Evaluate each combination to see if it equals 24. [This step is necessary to check if the combination is a valid solution to the problem.] 
 If a combination equals 24, then that is the solution. If no combination equals 24, then the problem is unsolvable with the given numbers. [This step is necessary to determine the solution or conclude that there is no solution.] 
 
 
 
 
 
 [5] orchestrator 
 
 
 Tao, please proceed to work on the first step of the above plan. You must follow the response templates strictly 
so that your orchestrator can parse your response. Think carefully before responding. 
 
 
 
 
 [6] Tao 
 
 
 After trying different combinations of the four numbers using the four operations, I found that the following combination results in 24: 
 
 Multiply 6 and 2 to get 12. 
 Subtract 1 from 7 to get 6. 
 Multiply the results from steps 1 and 2 to get 24. 
 
 So, the solution is (6 * 2) * (7 - 1) = 24. 
 
 
 
 
 [7] Tao 
 
 
 The solution to the Game of 24 problem with the numbers 7, 6, 2, 1 is to multiply 6 and 2 to get 12, subtract 1 from 7 to get 6, and then multiply these two results to get 24. So, the solution is (6 * 2) * (7 - 1) = 24. 
 
 
 total tokens : 3683